In [37]:
# Получаем список городов, на которых суммарный вес превышает вместимость грузовика
def subtourslice(tour, vehicle, max_vehicle):
    #print("--------------------------------Начало работы функции--------------------------------")
    #vehilce[i][0] - номер города, vehicle[i][1] - вес груза для перевозки
    capacity_used = np.zeros(len(vehicle))
    k = 0
    slice = []
    for i in range(len(vehicle)):
        # capacity_used[i] - начальный вес
        while((capacity_used[i] <= max_vehicle) and (k <= (len(tour) - 1))):
            capacity_used[i] += vehicle[k][1] # заполняем грузовик конкретным кол-вом груза для каждого города
            if(capacity_used[i] > max_vehicle):
                # если кол-во груза, который нужно отвезти, больше чем вместимость грузовика, то вычитаем это кол-во груза
                capacity_used[i] -= vehicle[k][1]
                k -= 1
                slice.append(vehicle[k][0]) # запоминаем номер тура, чтобы повторно не проходить по такому же маршруту
                k += 1
                break
            k += 1
    slice.append(vehicle[k-1][0])
    return(slice)

def subtour(slice, tour):
    sub = []
    # добавляем номера городов, после которых суммарный вес груза превышал объем грузовика
    p = tour[:(slice[0] + 1)]
    p.append(tour[0])
    if(0 not in p):
        h = randint(1, len(p)-1)
        p.insert(h, 0)
    sub.append(p)
    for i in range(0, len(slice) - 1):
        # нарежем маршруты городов до момента, пока суммарный вес груза не превысел объем грузовика
        k = tour[(slice[i] + 1):(slice[i + 1] + 1)] 
        k.append(tour[(slice[i] + 1)])
        if(0 not in k):
            h = randint(1, len(k)-1)
            k.insert(h, 0)
        sub.append(k)
    return sub

In [38]:
def make_valid_tour(tour, tour_edges, X, Y, distance, num_cities):
    if len(Y) - len(X) != 0:
        return [], False
    tour_edges_new = copy.deepcopy(tour_edges)
    for i in range(len(X)):
        Xii = list(X[i])
        tour_edges_new[Xii[0]].remove(Xii[1])
        tour_edges_new[Xii[1]].remove(Xii[0])
        Yii = list(Y[i])
        tour_edges_new[Yii[0]].add(Yii[1])
        tour_edges_new[Yii[1]].add(Yii[0])
    for i in tour_edges_new:
        if len(tour_edges_new[i]) != 2:
            return [], False
    new_tour = ['tour',0]
    i = list(tour_edges_new[0])[0]
    new_tour += [i]
    for foo in range(num_cities-1):
        node_connections = list(tour_edges_new[i]) # всевозможные ребра
        for j in [0,1]:
            if node_connections[j] != new_tour[-2]:
                new_tour += [node_connections[j]]
                if node_connections[j] == new_tour[1]:
                    new_tour.pop(0)
                    if len(new_tour) == num_cities+1:
                        return [new_tour, tour_edges_new], True
                    else:
                        return [], False
                i = node_connections[j]
                break
    print('Error')

    
def find_edge_to_remove(tour, tour_edges, delta, broken_edges, created_edges, initial_node, latest_node, distance, num_cities):
    if(latest_node in tour):
        j = tour.index(latest_node)
    else:
        return False, [], {}
    for t in neighbours(j, tour, num_cities):
        xi = {latest_node, t}
        if xi not in created_edges and xi not in broken_edges and t != initial_node:
            Xi = broken_edges[:]
            Xi += [xi]
            k = list(xi)
            reconnecting_edge = {t, initial_node}
            tempY = created_edges[:]
            tempY += [reconnecting_edge]
            new_tour_object, result = make_valid_tour(tour, tour_edges, Xi, tempY, distance, num_cities)
            if result == True:
                temp_delta = delta - distance[(k[0], k[1])]
                if temp_delta + distance[(t, initial_node)] < 0:
                    return True, new_tour_object[0], new_tour_object[1]
                else:
                    return find_edge_to_add(tour, tour_edges, temp_delta, Xi, created_edges, initial_node, t, distance, num_cities)
    return False, [], {}

def find_edge_to_add(tour, tour_edges, delta, broken_edges, created_edges, initial_node, latest_node, distance, num_cities):
    for t in range(num_cities):
        if t not in neighbours(latest_node, tour, num_cities) + [latest_node]:
            yi = {latest_node, t}
            temp_delta = delta + distance[(latest_node, t)]
            if yi not in broken_edges and temp_delta < 0:
                tempY = created_edges[:]
                tempY += [yi]
                return find_edge_to_remove(tour, tour_edges, temp_delta, broken_edges, tempY, initial_node, t, distance, num_cities)
    return False, [], {}

def neighbours(i, tour, num_cities):
    if i > 0 and i < num_cities:
        return [tour[i-1], tour[i+1]]
    else:
        return [tour[1], tour[-2]]

def apply_LK(tour, tour_edges, start, distance, num_cities, old_tour_length):
    min_tour_length = old_tour_length * 10
    min_tour = False
    for i in range(num_cities):
        if time.time() - start > 100:
            if min_tour:
                return False, min_tour, min_tour_edges
            else:
                return False, tour, tour_edges
        t1 = tour[i]
        for t2 in neighbours(i, tour, num_cities):
            x1 = {t1, t2}
            X = [x1]
            for j in tour:
                if j not in x1:
                    t3 = j
                    y1 = {t2, t3}
                    delta1 = distance[(t2, t3)] - distance[(t1, t2)]
                    Y = [y1]
                    if delta1 < 0:
                        result = find_edge_to_remove(tour, tour_edges, delta1, X, Y, t1, t3, distance, num_cities)
                        if result[0]:
                            temp_tour_length = 0
                            temp_tour = result[1]
                            for i in range(0,num_cities-1):
                                temp_tour_length = temp_tour_length + distance[(temp_tour[i], temp_tour[i+1])]
                            temp_tour_length = temp_tour_length + distance[(temp_tour[num_cities-1], temp_tour[0])]
                            if temp_tour_length < min_tour_length:
                                min_tour_length = temp_tour_length
                                min_tour = temp_tour
                                min_tour_edges = result[2]
    if min_tour:
        return True, min_tour, min_tour_edges
    return False, tour, {}

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import operator
from functools import reduce
import itertools
import random
from random import randint
import copy

#set up for the loop calling LK
n = 101

cities = [i for i in range(n)] # список номеров городов
print(cities)

roads = [(i, j) for i in range(n) for j in range(n) if(i != j)] # это попарные маршруты между всеми городами

#np.random.seed(0)
x = np.random.rand(n) * 10
y = np.random.rand(n) * 10

distance = {(i,j):np.hypot(x[i] - x[j], y[i]-y[j]) for i,j in roads}

old_tour_length = 0

tour = list(range(n))
random.shuffle(tour)
print("TOUR", tour)

min_capasity = 1
max_capasity = 42
max_vehicle = 50
capacity = np.minimum(np.maximum(np.abs(np.random.normal(15, 10, size=[1, n])), min_capasity), max_capasity)

p = 1
max_vehicle *= p
while(max_vehicle < max_capasity):
    max_vehicle = max_vehicle / p
    p += 1
    max_vehicle *= p
print("Используется грузовиков: ", p)

q = []
capacity[0][0] = 0
for i in range(0, len(tour)):
    q.append((i, capacity[0][i])) # вес груза, который необходимо перевести в город

lst_subtour = subtourslice(tour, q, max_vehicle)
all_subtour = subtour(lst_subtour, tour)
print("all_subtour", all_subtour)

dist_subtour = []
s = 0
for p in range(0, len(all_subtour)):
    for i in range(len(all_subtour[p])-1):
        s += distance[(all_subtour[p][i], all_subtour[p][i+1])]
    dist_subtour.append(s)
    s = 0
print("Old distance", sum(dist_subtour))

all_tour = []
for p in range(0, len(all_subtour)):
    success = True
    start=time.time()
    tour_edges = {}
    for i in range(1, len(all_subtour[p])-1):
        tour_edges[all_subtour[p][i]] = {all_subtour[p][i+1], all_subtour[p][i-1]}
    tour_edges[all_subtour[p][0]] = {all_subtour[p][1], all_subtour[p][-2]}
    if(len(all_subtour[p]) > 3):
        while success and time.time() - start < 100:
            success, all_subtour[p], tour_edges = apply_LK(all_subtour[p], tour_edges, start, distance, len(all_subtour[p])-1, dist_subtour[p])
    #print("Tour", all_subtour[p])
    all_tour.append(all_subtour[p])

s = 0
for p in range(0, len(all_tour)):
    for i in range(len(all_tour[p])-1):
        if(len(all_tour[p]) != 2 and (all_tour[p][i] != all_tour[p][i+1])):
            s += distance[(all_tour[p][i], all_tour[p][i+1])]
    all_subtour[p].pop(-1)
print("New distance", s)
    
print(all_tour)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
TOUR [94, 4, 70, 6, 91, 69, 58, 65, 47, 28, 61, 93, 34, 80, 57, 23, 82, 37, 85, 43, 66, 86, 89, 13, 32, 24, 81, 20, 19, 29, 87, 60, 36, 26, 56, 75, 79, 68, 51, 92, 11, 33, 72, 63, 83, 64, 27, 76, 55, 14, 99, 73, 40, 62, 78, 31, 77, 97, 12, 22, 100, 16, 30, 74, 15, 3, 50, 54, 35, 45, 9, 90, 5, 8, 0, 84, 1, 18, 49, 98, 25, 42, 52, 2, 44, 7, 17, 41, 48, 21, 88, 67, 96, 53, 46, 38, 95, 71, 59, 10, 39]
Используется грузовиков:  1
all_subtour [[94, 4, 70, 0, 94], [6, 91, 69, 58, 0, 6], [65, 47, 0, 28, 65], [61, 93, 34, 0, 61], [80, 0, 80], [57, 0, 57], [23, 82, 0, 37, 23], [85, 43, 0, 85], [66, 0, 86, 6